In [ ]:
%run utils.ipynb
%run seq2seq.ipynb
%run seq2seq_with_attention.ipynb
%run effective_approaches.ipynb
%run coverage.ipynb

In [ ]:
torch.cuda.set_device(0)

In [ ]:
d = Dataset(source_files, target_files, batch_size=10)
embedding_dim = 50
hidden_dim = 100
num_layers = 2

In [ ]:
def train_and_save(m, d, str_, coverage=False, coverage_type="linguistic", log=False):
    prev_score = -1
    for i in range(30):
        train(m, d, coverage, coverage_type, log=log)
        score = eval(m, d)
        best_state = None
        print()
        print(score)
        if score > prev_score:
            #torch.save(m.state_dict(), str_ + str(score)[:4] + ".params_")
            best_state = m.state_dict()
            prev_score = score
            
    torch.save(best_state, str_ + str(prev_score)[:6] + ".params")
    print(prev_score)

In [ ]:
m = Seq2Seq(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start)
print(m.cuda())
train_and_save(m, d, "seq2seq_")

In [ ]:
m = Seq2SeqWithAttention(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start)
print(m.cuda())
train_and_save(m, d, "attention_")

In [ ]:
m = Seq2Seq_EffectiveApproaches(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start,
            Attention=Attention_General)
print(m.cuda())
train_and_save(m, d, "eff_attn_gen_")

In [ ]:
m = Seq2Seq_EffectiveApproaches(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start,
            Attention=Attention_Dot)
print(m.cuda())
train_and_save(m, d, "eff_attn_dot_")

In [ ]:
m = Seq2Seq_EffectiveApproaches(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start,
            Attention=Attention_Concat)
print(m.cuda())
train_and_save(m, d, "eff_attn_concat_")

In [ ]:
# also works with attn dot and attn concat
m = Seq2Seq_Coverage(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start,
            Attention=Attention_General)
print(m.cuda())
train_and_save(m, d, "coverage_attn_gen_ling_", True, "linguistic")

In [ ]:
# also works with attn dot and attn concat
m = Seq2Seq_Coverage(d.len_source, d.len_target, embedding_dim, hidden_dim, num_layers=2, 
            padding_dim_source=d.source_pad, padding_dim_target=d.target_pad, start_dim_target=d.target_start,
            Attention=Attention_General)
print(m.cuda())
train_and_save(m, d, "coverage_attn_gen_fert_", True, "fertility")